<a href="https://colab.research.google.com/github/RaxFord1/botreplacement/blob/master/CartPole_V0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To save video, we need X screen, suddenly colab is a cloud-service and we don't have screen at all, so we need to simulate it with xvfb lib

In [0]:
!apt-get install -y xvfb python-opengl > /dev/null 2>&1
!pip install gym pyvirtualdisplay > /dev/null 2>&1

import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

Creating virtual screen

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(400, 300))
display.start()

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1009'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '400x300x24', ':1009'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

**Importing all needs `gym, pytorch, numpy`**

In [0]:
import gym
from collections import namedtuple
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

***Setings for our NN***

In [0]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

Our Neural Network, here we have Sequential structure with 2 Linear **layers**. Input defined by our environment's observation and output defined by agent's number of actions.

In [0]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

Creating our own classes "**Episode**" and "**EpisodeStep**". ***EpisodeStep*** class will contain each *`observation`* and token *`action`*. ***Episode*** will contain all *`EpisodeStep`* records and it's `earned *total rewards*`

In [0]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

Method iterate_bathces takes positional arguments: our environment,  our neural network and batch_size we want to get. The output is list of **` Episode`** class with random actions

In [0]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

Method filter_batch takes positional arguments: **batches** and **percintile**. Here we define *mean reward* of all batch,  %*percentile* of all given trials . Filters all results and return only best (>percentile). Returns **observations** , **acts**, **%percentile**, **mean reward**

In [0]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

Creating **environment** for training. Define ***`number`***  of **actions** and **observations** for this game

In [0]:
env = gym.make("CartPole-v0")
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

Defining our **NN** with its **optimizer** and **activator** functions

In [0]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
sm = nn.Softmax(dim=1)

Training our model for the best result (win is 195, but maximum actions we can do is 200, so until we get reward equal 200)

In [11]:
for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    #all gradients = 0
    optimizer.zero_grad()
    #getting our predictions(2 vals) with given observations
    action_scores_v = net(obs_v)
    #Measuring losses between predictions and best acts we get  
    loss_v = objective(action_scores_v, acts_v)
    #measuring all gradients
    loss_v.backward()
    #optimising our NN
    optimizer.step()
    #printing info each episode
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    if reward_m > 199:
        print("Solved!")
        break

0: loss=0.701, reward_mean=20.5, reward_bound=23.0
1: loss=0.660, reward_mean=16.9, reward_bound=19.0
2: loss=0.655, reward_mean=17.8, reward_bound=19.0
3: loss=0.687, reward_mean=19.4, reward_bound=24.5
4: loss=0.672, reward_mean=21.5, reward_bound=22.5
5: loss=0.681, reward_mean=19.1, reward_bound=21.5
6: loss=0.663, reward_mean=26.5, reward_bound=28.5
7: loss=0.657, reward_mean=30.7, reward_bound=38.5
8: loss=0.648, reward_mean=41.9, reward_bound=43.5
9: loss=0.634, reward_mean=38.1, reward_bound=49.5
10: loss=0.622, reward_mean=35.6, reward_bound=43.0
11: loss=0.631, reward_mean=35.4, reward_bound=34.0
12: loss=0.625, reward_mean=35.9, reward_bound=41.0
13: loss=0.615, reward_mean=41.1, reward_bound=44.0
14: loss=0.611, reward_mean=44.5, reward_bound=46.5
15: loss=0.587, reward_mean=44.7, reward_bound=56.0
16: loss=0.589, reward_mean=52.4, reward_bound=58.5
17: loss=0.576, reward_mean=55.2, reward_bound=58.0
18: loss=0.593, reward_mean=62.6, reward_bound=71.0
19: loss=0.575, reward

Creating our environment with screen for **recording**

In [0]:
env = gym.wrappers.Monitor(env, directory="mon", force=True)

Method play_n_times makes a trials, where we deploy our NN. Positional args: Environment and number of trials. Returns only reward from each trial.

In [0]:
def play_n_times(env, net, n):
  history = list()
  
  for i in range(n):
    rewards = 0
    obs = env.reset()
    done = False
    
    while not done:
      #NN gets only tensor-like information
      obs = torch.FloatTensor([obs])
      #getting predictions from our NN
      probs = net(obs)
      #using activation layer on results
      act_probs_v = sm(probs)
      #converting to numpy data-type
      act_probs_v.data.numpy()[0]
      act_probs = act_probs_v.data.numpy()[0]
      #getting index of highest prediction
      act_probs.max()
      if act_probs.max()== act_probs[0]:
        action = 0
      else:
        action = 1
      next_obs, reward, is_done, _ = env.step(action)
      rewards += reward
      done = is_done
      obs = next_obs
      
    history.append(rewards)
    
  return history

Starting our agent for interacting with environment.

In [0]:
history = play_n_times(env, net, 10)
history = np.array(history)

Analyzing data

In [15]:
minimum = history.min()
maximum = history.max()
mean = history.mean()
print("Minimal agent's performance = %d; Maximum agent's performance = %d; Mean agent's performance = %d; Extra = %d" % (minimum, maximum, mean, (minimum+maximum)/(2*mean)))

Minimal agent's performance = 200; Maximum agent's performance = 200; Mean agent's performance = 200; Extra = 1
